In [1]:
from transformers import AutoModel, AutoTokenizer
phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
! pip install rouge --quiet

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import json
import numpy as np
import pandas as pd
from rouge import Rouge
import string
import copy
import time
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import LatentDirichletAllocation

def getRouge2(ref, pred, kind): # tokenized input
    try:
        return round(Rouge().get_scores(pred.lower(), ref.lower())[0]['rouge-2'][kind], 4)
    except ValueError:
        return 0.0
def getRouge1(ref, pred, kind): # tokenized input
    return Rouge().get_scores(pred.lower(), ref.lower())[0]['rouge-1'][kind]
def getRougeL(ref, pred, kind): # tokenized input
    return Rouge().get_scores(pred.lower(), ref.lower())[0]['rouge-l'][kind]

In [4]:
# class MLP(nn.Module):
#     def __init__(self, in_dim, out_dim, hid_dim, layers=2, act=nn.LeakyReLU(), dropout_p=0.3, keep_last_layer=False):
#         super(MLP, self).__init__()
#         self.layers = layers
#         self.act = act
#         self.dropout = nn.Dropout(dropout_p)
#         self.keep_last = keep_last_layer

#         self.mlp_layers = nn.ModuleList([])
#         if layers == 1:
#             self.mlp_layers.append(nn.Linear(in_dim, out_dim))
#         else:
#             self.mlp_layers.append(nn.Linear(in_dim, hid_dim))
#             for i in range(self.layers - 2):
#                 self.mlp_layers.append(nn.Linear(hid_dim, hid_dim))
#             self.mlp_layers.append(nn.Linear(hid_dim, out_dim))

#     def forward(self, x):
#         for i in range(len(self.mlp_layers) - 1):
#             x = self.dropout(self.act(self.mlp_layers[i](x)))
#         if self.keep_last:
#             x = self.mlp_layers[-1](x)
#         else:
#             x = self.act(self.mlp_layers[-1](x))
#         return x


# borrowed from labml.ai
# class GraphAttentionLayer(nn.Module):
#     def __init__(self, in_features: int, out_features: int, n_heads: int,
#                  is_concat: bool = True,
#                  dropout: float = 0.6,
#                  leaky_relu_negative_slope: float = 0.2):
#         """
#         * `in_features`, $F$, is the number of input features per node
#         * `out_features`, $F'$, is the number of output features per node
#         * `n_heads`, $K$, is the number of attention heads
#         * `is_concat` whether the multi-head results should be concatenated or averaged
#         * `dropout` is the dropout probability
#         * `leaky_relu_negative_slope` is the negative slope for leaky relu activation
#         """
#         super().__init__()

#         self.is_concat = is_concat
#         self.n_heads = n_heads

#         # Calculate the number of dimensions per head
#         if is_concat:
#             assert out_features % n_heads == 0
#             # If we are concatenating the multiple heads
#             self.n_hidden = out_features // n_heads
#         else:
#             # If we are averaging the multiple heads
#             self.n_hidden = out_features

#         # Linear layer for initial transformation;
#         # i.e. to transform the node embeddings before self-attention
#         self.linear = nn.Linear(in_features, self.n_hidden * n_heads, bias=False)
#         # Linear layer to compute attention score $e_{ij}$
#         self.attn = nn.Linear(self.n_hidden * 2, 1, bias=False)
#         # The activation for attention score $e_{ij}$
#         self.activation = nn.LeakyReLU(negative_slope=leaky_relu_negative_slope)
#         # Softmax to compute attention $\alpha_{ij}$
#         self.softmax = nn.Softmax(dim=1)
#         # Dropout layer to be applied for attention
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, h: torch.Tensor, adj_mat: torch.Tensor, docnum, secnum):
#         """
#         * `h`, $\mathbf{h}$ is the input node embeddings of shape `[n_nodes, in_features]`.
#         * `adj_mat` is the adjacency matrix of shape `[n_nodes, n_nodes, n_heads]`.
#         We use shape `[n_nodes, n_nodes, 1]` since the adjacency is the same for each head.

#         Adjacency matrix represent the edges (or connections) among nodes.
#         `adj_mat[i][j]` is `True` if there is an edge from node `i` to node `j`.
#         """

#         # Number of nodes
#         n_nodes = h.shape[0]
#         g = self.linear(h).view(n_nodes, self.n_heads, self.n_hidden)
#         g_repeat = g.repeat(n_nodes, 1, 1)
#         g_repeat_interleave = g.repeat_interleave(n_nodes, dim=0)
#         g_concat = torch.cat([g_repeat_interleave, g_repeat], dim=-1)
#         g_concat = g_concat.view(n_nodes, n_nodes, self.n_heads, 2 * self.n_hidden)
#         e = self.activation(self.attn(g_concat))
#         # del g_concat, g_repeat, g_repeat_interleave
#         # torch.cuda.empty_cache()
#         # Remove the last dimension of size `1`
#         e = e.squeeze(-1)

#         # The adjacency matrix should have shape
#         # `[n_nodes, n_nodes, n_heads]` or`[n_nodes, n_nodes, 1]`
#         assert adj_mat.shape[0] == 1 or adj_mat.shape[0] == n_nodes
#         assert adj_mat.shape[1] == 1 or adj_mat.shape[1] == n_nodes
#         assert adj_mat.shape[2] == 1 or adj_mat.shape[2] == self.n_heads
#         # Mask $e_{ij}$ based on adjacency matrix.
#         # $e_{ij}$ is set to $- \infty$ if there is no edge from $i$ to $j$.
#         e = e.masked_fill(adj_mat == 0, float(-1e9))
#         a = self.softmax(e)
#         a = self.dropout(a)
#         attn_res = torch.einsum('ijh,jhf->ihf', a, g)

#         # Concatenate the heads
#         if self.is_concat:
#             return attn_res.reshape(n_nodes, self.n_heads * self.n_hidden)
#         # Take the mean of the heads
#         else:
#             return attn_res.mean(dim=1)


# class GAT(nn.Module):
#     def __init__(self, in_features: int, n_hidden: int, n_classes: int, n_heads: int, dropout: float):
#         super().__init__()
#         self.layer1 = GraphAttentionLayer(in_features, n_hidden, n_heads, is_concat=True, dropout=dropout)
#         self.activation = nn.ELU()
#         self.output = GraphAttentionLayer(n_hidden, n_classes, 1, is_concat=False, dropout=dropout)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x: torch.Tensor, adj_mat: torch.Tensor, docnum, secnum):
#         x = x.squeeze(0)
#         adj_mat = adj_mat.squeeze(0)
#         # adj_x = adj_mat.clone().sum(dim=1, keepdim=True).repeat(1, x.shape[1]).bool()
#         adj_mat = adj_mat.unsqueeze(-1).bool()
#         x = self.dropout(x)
#         x = self.layer1(x, adj_mat, docnum, secnum)
#         x = self.activation(x)
#         x = self.dropout(x)
#         x = self.output(x, adj_mat, docnum, secnum).masked_fill(adj_x == 0, float(0))
#         return x.unsqueeze(0)


# class StepWiseGraphConvLayer(nn.Module):
#     def __init__(self, in_dim, out_dim, hid_dim, dropout_p=0.3, act=nn.LeakyReLU(), nheads=6, iter=1, final="att"):
#         super().__init__()
#         self.act = act
#         self.dropout = nn.Dropout(dropout_p)
#         self.iter = iter
#         self.gat = nn.ModuleList([GAT(in_features=in_dim, n_hidden=hid_dim, n_classes=in_dim,
#                                       dropout=dropout_p, n_heads=nheads) for _ in range(iter)])
#         self.gat2 = nn.ModuleList([GAT(in_features=in_dim, n_hidden=hid_dim, n_classes=in_dim,
#                                        dropout=dropout_p, n_heads=nheads) for _ in range(iter)])
#         self.gat3 = nn.ModuleList([GAT(in_features=in_dim, n_hidden=hid_dim, n_classes=in_dim,
#                                        dropout=dropout_p, n_heads=nheads) for _ in range(iter)])

#         self.feature_fusion_layer = nn.Linear(in_dim * 3, in_dim)
#         self.ffn = MLP(in_dim, in_dim, hid_dim, dropout_p=dropout_p, layers=3)
#         self.out_ffn = MLP(in_dim, in_dim, hid_dim, dropout_p=dropout_p)

#     def forward(self, feature, adj, docnum, secnum):
#         sen_adj = adj.clone()
#         sen_adj[:, :, -docnum - secnum - 1:] = 0
#         sec_adj = adj.clone()
#         sec_adj[:, :, :-docnum - secnum - 1] = sec_adj[:, :, -docnum - 1:] = 0
#         doc_adj = adj.clone()
#         doc_adj[:, :, :-docnum - 1] = 0

#         feature_sen = feature.clone()
#         feature_sec = feature.clone()
#         feature_doc = feature.clone()
#         feature_resi = feature

#         feature_sen_re = feature_sen
#         feature_sec_re = feature_sec
#         feature_doc_re = feature_doc
#         for i in range(0, self.iter):
#             feature_sen = self.gat[i](feature_sen, sen_adj)
#         feature_sen += feature_sen_re

#         for i in range(0, self.iter):
#             feature_sec = self.gat2[i](feature_sec, sec_adj)
#         feature_sec += feature_sec_re

#         for i in range(0, self.iter):
#             feature_doc = self.gat3[i](feature_doc, doc_adj)
#         feature_doc += feature_doc_re

#         feature = torch.concat([feature_doc, feature_sec, feature_sen], dim=-1)
#         feature = self.dropout(F.leaky_relu(self.feature_fusion_layer(feature)))
#         feature = self.ffn(feature)
#         feature = self.out_ffn(feature) + feature_resi
#         return feature

In [5]:
class MLP(nn.Module):
    def __init__(self, in_dim, out_dim, hid_dim, layers=2, act=nn.LeakyReLU(), dropout_p=0.3, keep_last_layer=False):
        super(MLP, self).__init__()
        self.layers = layers
        self.act = act
        self.dropout = nn.Dropout(dropout_p)
        self.keep_last = keep_last_layer

        self.mlp_layers = nn.ModuleList([])
        if layers == 1:
            self.mlp_layers.append(nn.Linear(in_dim, out_dim))
        else:
            self.mlp_layers.append(nn.Linear(in_dim, hid_dim))
            for i in range(self.layers - 2):
                self.mlp_layers.append(nn.Linear(hid_dim, hid_dim))
            self.mlp_layers.append(nn.Linear(hid_dim, out_dim))

    def forward(self, x):
        for i in range(len(self.mlp_layers) - 1):
            x = self.dropout(self.act(self.mlp_layers[i](x)))
        if self.keep_last:
            x = self.mlp_layers[-1](x)
        else:
            x = self.act(self.mlp_layers[-1](x))
        return x


# borrowed from labml.ai
class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features: int, out_features: int, n_heads: int,
                 is_concat: bool = True,
                 dropout: float = 0.6,
                 leaky_relu_negative_slope: float = 0.2):
        """
        * `in_features`, $F$, is the number of input features per node
        * `out_features`, $F'$, is the number of output features per node
        * `n_heads`, $K$, is the number of attention heads
        * `is_concat` whether the multi-head results should be concatenated or averaged
        * `dropout` is the dropout probability
        * `leaky_relu_negative_slope` is the negative slope for leaky relu activation
        """
        super().__init__()

        self.is_concat = is_concat
        self.n_heads = n_heads

        # Calculate the number of dimensions per head
        if is_concat:
            assert out_features % n_heads == 0
            # If we are concatenating the multiple heads
            self.n_hidden = out_features // n_heads
        else:
            # If we are averaging the multiple heads
            self.n_hidden = out_features

        # Linear layer for initial transformation;
        # i.e. to transform the node embeddings before self-attention
        self.linear = nn.Linear(in_features, self.n_hidden * n_heads, bias=False)
        # Linear layer to compute attention score $e_{ij}$
        self.attn = nn.Linear(self.n_hidden * 2, 1, bias=False)
        # The activation for attention score $e_{ij}$
        self.activation = nn.LeakyReLU(negative_slope=leaky_relu_negative_slope)
        # Softmax to compute attention $\alpha_{ij}$
        self.softmax = nn.Softmax(dim=1)
        # Dropout layer to be applied for attention
        self.dropout = nn.Dropout(dropout)

    def forward(self, h: torch.Tensor, adj_mat: torch.Tensor):
        """
        * `h`, $\mathbf{h}$ is the input node embeddings of shape `[n_nodes, in_features]`.
        * `adj_mat` is the adjacency matrix of shape `[n_nodes, n_nodes, n_heads]`.
        We use shape `[n_nodes, n_nodes, 1]` since the adjacency is the same for each head.

        Adjacency matrix represent the edges (or connections) among nodes.
        `adj_mat[i][j]` is `True` if there is an edge from node `i` to node `j`.
        """

        # Number of nodes
        n_nodes = h.shape[0]
        g = self.linear(h).view(n_nodes, self.n_heads, self.n_hidden)
        g_repeat = g.repeat(n_nodes, 1, 1)
        g_repeat_interleave = g.repeat_interleave(n_nodes, dim=0)
        g_concat = torch.cat([g_repeat_interleave, g_repeat], dim=-1)
        g_concat = g_concat.view(n_nodes, n_nodes, self.n_heads, 2 * self.n_hidden)
        e = self.activation(self.attn(g_concat))
        # del g_concat, g_repeat, g_repeat_interleave
        # torch.cuda.empty_cache()
        # Remove the last dimension of size `1`
        e = e.squeeze(-1)

        # The adjacency matrix should have shape
        # `[n_nodes, n_nodes, n_heads]` or`[n_nodes, n_nodes, 1]`
        assert adj_mat.shape[0] == 1 or adj_mat.shape[0] == n_nodes
        assert adj_mat.shape[1] == 1 or adj_mat.shape[1] == n_nodes
        assert adj_mat.shape[2] == 1 or adj_mat.shape[2] == self.n_heads
        # Mask $e_{ij}$ based on adjacency matrix.
        # $e_{ij}$ is set to $- \infty$ if there is no edge from $i$ to $j$.
        e = e.masked_fill(adj_mat == 0, float(-1e9))
        a = self.softmax(e)
        a = self.dropout(a)
        attn_res = torch.einsum('ijh,jhf->ihf', a, g)

        # Concatenate the heads
        if self.is_concat:
            return attn_res.reshape(n_nodes, self.n_heads * self.n_hidden)
        # Take the mean of the heads
        else:
            return attn_res.mean(dim=1)


class GAT(nn.Module):
    def __init__(self, in_features: int, n_hidden: int, n_classes: int, n_heads: int, dropout: float):
        super().__init__()
        self.layer1 = GraphAttentionLayer(in_features, n_hidden, n_heads, is_concat=True, dropout=dropout)
        self.activation = nn.ELU()
        self.output = GraphAttentionLayer(n_hidden, n_classes, 1, is_concat=False, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, adj_mat: torch.Tensor):
        x = x.squeeze(0)
        adj_mat = adj_mat.squeeze(0)
        adj_mat = adj_mat.unsqueeze(-1).bool()
        x = self.dropout(x)
        #         print('inp', x)
        x = self.layer1(x, adj_mat)
        #         print(x)
        x = self.activation(x)
        x = self.dropout(x)
        return self.output(x, adj_mat).unsqueeze(0)


class StepWiseGraphConvLayer(nn.Module):
    def __init__(self, in_dim, out_dim, hid_dim, dropout_p=0.3, act=nn.LeakyReLU(), nheads=6, iter=1, final="att"):
        super().__init__()
        self.act = act
        self.dropout = nn.Dropout(dropout_p)
        self.iter = iter
        self.gat = nn.ModuleList([GAT(in_features=in_dim, n_hidden=hid_dim, n_classes=in_dim,
                                      dropout=dropout_p, n_heads=nheads) for _ in range(iter)])
        self.gat2 = nn.ModuleList([GAT(in_features=in_dim, n_hidden=hid_dim, n_classes=in_dim,
                                       dropout=dropout_p, n_heads=nheads) for _ in range(iter)])
        self.gat3 = nn.ModuleList([GAT(in_features=in_dim, n_hidden=hid_dim, n_classes=in_dim,
                                       dropout=dropout_p, n_heads=nheads) for _ in range(iter)])

        self.feature_fusion_layer = nn.Linear(in_dim * 3, in_dim)
        self.ffn = MLP(in_dim, in_dim, hid_dim, dropout_p=dropout_p, layers=3)
        self.out_ffn = MLP(in_dim, in_dim, hid_dim, dropout_p=dropout_p)

    def forward(self, feature, adj, docnum, secnum):
        sen_adj = adj.clone()
        sen_adj[:, :, -docnum - secnum - 1:] = 0
        sec_adj = adj.clone()
        sec_adj[:, :, :-docnum - secnum - 1] = sec_adj[:, :, -docnum - 1:] = 0
        doc_adj = adj.clone()
        doc_adj[:, :, :-docnum - 1] = 0

        feature_sen = feature.clone()
        feature_sec = feature.clone()
        feature_doc = feature.clone()
        feature_resi = feature

        feature_sen_re = feature_sen
        feature_sec_re = feature_sec
        feature_doc_re = feature_doc
        for i in range(0, self.iter):
            feature_sen = self.gat[i](feature_sen, sen_adj)
        feature_sen += feature_sen_re

        for i in range(0, self.iter):
            feature_sec = self.gat2[i](feature_sec, sec_adj)
        feature_sec += feature_sec_re

        for i in range(0, self.iter):
            feature_doc = self.gat3[i](feature_doc, doc_adj)
        feature_doc += feature_doc_re

        feature = torch.concat([feature_doc, feature_sec, feature_sen], dim=-1)
        feature = self.dropout(F.leaky_relu(self.feature_fusion_layer(feature)))
        feature = self.ffn(feature)
        feature = self.out_ffn(feature) + feature_resi
        return feature


In [6]:
class Contrast_Encoder(nn.Module):
    def __init__(self, graph_encoder, hidden_dim, bert_hidden=768, in_dim=768, dropout_p=0.3):
        super(Contrast_Encoder, self).__init__()
        self.graph_encoder = graph_encoder
        self.common_proj_mlp = MLP(in_dim, in_dim, hidden_dim, dropout_p=dropout_p, act=nn.LeakyReLU())

    def forward(self, p_gfeature, p_adj, docnum, secnum):
        pg = self.graph_encoder(p_gfeature.float(), p_adj.float(), docnum, secnum)
        pg = self.common_proj_mlp(pg)
        return pg


class End2End_Encoder(nn.Module):
    def __init__(self, graph_encoder, in_dim, hidden_dim, dropout_p):
        super(End2End_Encoder, self).__init__()
        self.graph_encoder = graph_encoder
        self.dropout = nn.Dropout(dropout_p)
        self.out_proj_layer_mlp = MLP(in_dim, in_dim, hidden_dim, act=nn.LeakyReLU(), dropout_p=dropout_p, layers=2)
        self.final_layer = nn.Linear(in_dim, 1)

    def forward(self, x, adj, docnum, secnum):
        x = self.graph_encoder(x.float(), adj.float(), docnum, secnum)
        x = x[:, :-docnum-secnum-1, :]
        x = self.out_proj_layer_mlp(x)
        x = self.final_layer(x)
        return x

In [7]:
def _similarity(h1: torch.Tensor, h2: torch.Tensor):
    h1 = F.normalize(h1)
    h2 = F.normalize(h2)
    return h1 @ h2.t()

class InfoNCE(nn.Module):
    def __init__(self, tau):
        super(InfoNCE, self).__init__()
        self.tau = tau

    def forward(self, anchor, sample, pos_mask, neg_mask, *args, **kwargs):
        sim = _similarity(anchor, sample) / self.tau
        if len(anchor) > 1:
            sim, _ = torch.max(sim, dim=0)
        exp_sim = torch.exp(sim)
        loss = torch.log((exp_sim * pos_mask).sum(dim=1)) - torch.log((exp_sim * (pos_mask + neg_mask)).sum(dim=1))
        return -loss.mean()

In [8]:
def mask_to_adj(doc_sec_mask, sec_sen_mask):
    doc_sec_mask = np.array(doc_sec_mask)
    sec_sen_mask = np.array(sec_sen_mask)
    
    sen_num = sec_sen_mask.shape[1]
    sec_num = sec_sen_mask.shape[0]
    doc_num = doc_sec_mask.shape[0]
    adj = np.zeros((sen_num + sec_num + doc_num + 1, sen_num + sec_num + doc_num + 1))
    # section connection
    adj[-sec_num - doc_num - 1:-doc_num - 1, 0:-sec_num - doc_num - 1] = sec_sen_mask
    adj[0:-sec_num - doc_num - 1, -sec_num - doc_num - 1:-doc_num - 1] = sec_sen_mask.T
    for i in range(0, doc_num):
        doc_mask = doc_sec_mask[i]
        
        # Đảm bảo doc_mask là mảng numpy và có chiều đúng để reshape
        if doc_mask.ndim == 1:
            doc_mask = doc_mask.reshape((1, -1))
        elif doc_mask.ndim == 0:
            doc_mask = np.array([doc_mask])  # Chuyển thành mảng 1D nếu là số đơn lẻ
        
        adj[sen_num:-doc_num - 1, sen_num:-doc_num - 1] += doc_mask * doc_mask.T
        
    # doc connection
    adj[-doc_num - 1:-1, -sec_num - doc_num - 1:-doc_num - 1] = doc_sec_mask
    adj[-sec_num - doc_num - 1:-doc_num - 1, -doc_num - 1:-1] = doc_sec_mask.T
    adj[-doc_num - 1:, -doc_num - 1:] = 1

    # build sentence connection
    for i in range(0, sec_num):
        sec_mask = sec_sen_mask[i]
        
        # Đảm bảo sec_mask là mảng numpy và có chiều đúng để reshape
        if sec_mask.ndim == 1:
            sec_mask = sec_mask.reshape((1, -1))
        elif sec_mask.ndim == 0:
            sec_mask = np.array([sec_mask])  # Chuyển thành mảng 1D nếu là số đơn lẻ
        
        adj[:sen_num, :sen_num] += sec_mask * sec_mask.T
    return adj

In [9]:
def get_phoBert_vec(text, limit_len=400):
    sent = text.lower()
    input_ids = torch.tensor([tokenizer.encode(sent)])
    if input_ids.shape[1] > 256:
        # print('DEVIDED')
        sents = sent.split(' . ')
        wcnt = [len(s.split(' ')) for s in sents]
        wcnt_all = sum(wcnt)

        while wcnt_all > limit_len:
            # print('DEL')
            wcnt_all -= wcnt[-1]
            sents.pop()
            wcnt.pop()

        part1, part2 = [], []
        for i, s in enumerate(sents):
            if sum(wcnt[:i]) <= wcnt_all / 2:
                part1.append(s)
            else:
                part2.append(s)

        sents = [' . '.join(part1), ' . '.join(part2)]
        input_ids = [torch.tensor([tokenizer.encode(sent)]) for sent in sents]
        with torch.no_grad():
            return torch.cat([phobert(input_ids[0])["pooler_output"], phobert(input_ids[1])["pooler_output"]],
                             dim=0)

    with torch.no_grad():
        features = phobert(input_ids)
    return features["pooler_output"]

In [10]:
class Graph:
    def __init__(self, sents, sentVecs, scores, doc_sec_mask, sec_sen_mask, golden, threds=0.5):
        assert len(sentVecs) == len(scores) == len(sents)
        self.docnum = len(doc_sec_mask)
        self.secnum = len(sec_sen_mask)
        self.adj = torch.from_numpy(mask_to_adj(doc_sec_mask, sec_sen_mask)).float()
        self.feature = np.concatenate((np.array(sentVecs), np.zeros((self.secnum+self.docnum+1, sentVecs[0].size))))
        self.score = torch.from_numpy(np.array(scores)).float()
        self.score_onehot = (self.score >= threds).float()
        self.sents = np.array(sents)
        self.golden = golden
        self.goldenVec = get_phoBert_vec(golden)
        self.init_node_vec()
        self.feature = torch.from_numpy(self.feature).float()

    def init_node_vec(self):
        docnum, secnum = self.docnum, self.secnum
        for i in range(-secnum-docnum-1, -docnum-1):
            mask = self.adj[i].clone()
            mask[-secnum-docnum-1:] = 0
            self.feature[i] = np.mean(self.feature[mask.bool()], axis=0)
        for i in range(-docnum-1, -1):
            mask = self.adj[i].clone()
            mask[-docnum-1:] = 0
            self.feature[i] = np.mean(self.feature[mask.bool()], axis=0)
        self.feature[-1] = np.mean(self.feature[-docnum-1:-1], axis=0)

In [11]:
def meanTokenVecs(sent, sp=0):
    # spans = sent['spans']
    # if spans[sp]['vector'] is None:
    #     return
    # childVecs = [spans[sp]['vector']]
    # for child in spans[sp]['children']:
    #     if spans[child]['vector'] is not None:
    #         childVec = meanTokenVecs(sent, child)
    #         childVecs.append(childVec)
    # inp = np.mean(childVecs, axis=0)
    # return inp
    tokenVecList = [sp['vector'] for sp in sent['spans'] if sp['vector'] is not None]
    if not tokenVecList:
        print('Still error!')
        print(sent)
        return np.zeros(768)
    return np.mean(np.array(tokenVecList), axis=0)

def getPositionEncoding(pos, d=768, n=10000):
    P = np.zeros(d)
    for i in np.arange(int(d/2)):
        denominator = np.power(n, 2*i/d)
        P[2*i] = np.sin(pos/denominator)
        P[2*i+1] = np.cos(pos/denominator)
    return P

def divideIntoSections():
    secnum, sentnum = 0, 0
    for d, doc in enumerate(clusTree['docs']):
        total_words = 0
        sentnum += len(doc['sents'])
        for s, sent in enumerate(doc['sents']):
            total_words += len(sent['raw_sent'].split(' '))
        if total_words <= 500: minSec = 150
        elif total_words <= 1000: minSec = 200
        elif total_words <= 2000: minSec = 300
        else: minSec = 400

        wcnt, curOrgSecID, startSec = 0, 0, secnum
        for s, sent in enumerate(doc['sents']):
            wcnt += len(sent['raw_sent'].split(' '))
            clusTree['docs'][d]['sents'][s]['section_new'] = secnum
            if wcnt >= minSec and sent['secid'] != curOrgSecID and s < len(doc['sents']) - 1:
                secnum += 1
                wcnt = 0
            if sent['secid'] != curOrgSecID:
                curOrgSecID = sent['secid']

        if 0 < wcnt < minSec and secnum > startSec:
            for s in range(len(doc['sents']) - 1, -1, -1):
                if clusTree['docs'][d]['sents'][s]['section_new'] < secnum:
                    break
                clusTree['docs'][d]['sents'][s]['section_new'] = secnum - 1
            secnum -= 1
        secnum += 1
    return secnum, sentnum

stop_w = ['...']
with open('/kaggle/input/vlsp-dataset/vietnamese-stopwords-dash.txt', 'r', encoding='utf-8') as f:
    for w in f.readlines():
        stop_w.append(w.strip())
stop_w.extend([c for c in '!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~…“”’‘'])

with open('/kaggle/input/vlsp-dataset/LDA_models.pkl', mode='rb') as fp:
    cate_models = pickle.load(fp)

def removeRedundant(text):
    text = text.lower()
    words = [w for w in text.split(' ') if w not in stop_w]
    return ' '.join(words)

def divideIntoSections_lda():
    secnum, sentnum = 0, 0
    paraList, paras, ids, newSecID = [], [], [], {}
    for d, doc in enumerate(clusTree['docs']):
        sentnum += len(doc['sents'])
        paraList.append([])

        para, curOrgSecID = [], 0
        for s, sent in enumerate(doc['sents']):
            if sent['secid'] != curOrgSecID:
                paraList[-1].append(' '.join(para))
                para, curOrgSecID = [], sent['secid']
            para.append(sent['raw_sent'])
        if para is not []:
            paraList[-1].append(' '.join(para))

    for d, doc in enumerate(paraList):
        for p, para in enumerate(doc):
            paras.append(removeRedundant(para))
            ids.append((d, p))

    tf, lda_model = cate_models[clusTree['category']]
    X = tf.transform(paras)
    lda_top=lda_model.transform(X)

    secnum, groupset, wcnt = 0, {}, 0
    for p in range(len(paras)):
        idd = ids[p] # (doc, para)
        if min(lda_top[p]) == max(lda_top[p]):
            newSecID[(idd[0], idd[1])] = 0
            continue
        name, score = -1, 0
        for i,topic in enumerate(lda_top[p]):
            if topic > score:
                score, name = topic, i
        if name not in groupset:
            groupset[name] = len(groupset)
        newSecID[(idd[0], idd[1])] = groupset[name]

    prevSecnum, doc_endsec = 0, []
    for d, doc in enumerate(clusTree['docs']):
        groupset = {}
        for s, sent in enumerate(doc['sents']):
            if newSecID[(d, sent['secid'])] not in groupset:
                groupset[newSecID[(d, sent['secid'])]] = len(groupset) + prevSecnum
            clusTree['docs'][d]['sents'][s]['section_new'] = groupset[newSecID[(d, sent['secid'])]]
        prevSecnum = max(groupset.values()) + 1
        doc_endsec.append(max(groupset.values()))
    return doc_endsec, max(groupset.values()) + 1, sentnum

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your ow

In [12]:
with open('/kaggle/input/vlsp-graphs-sent/trainGraphs.pkl', 'rb') as fp:
    trainGraphs = pickle.load(fp)
with open('/kaggle/input/vlsp-graphs-sent/testGraphs.pkl', 'rb') as fp:
    testGraphs = pickle.load(fp)

/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [13]:
onehot = torch.cat([g.score_onehot for g in testGraphs], dim=-1)
onehot.shape, torch.sum(onehot)

(torch.Size([19350]), tensor(2033.))

In [14]:
trainGraphs = []
with open('/kaggle/input/vlsp-dataset/train_segmentedSumm.txt', 'r', encoding='utf-8') as f:
    goldenList = json.load(f)
for cluster in range(200):
    print(cluster)
    sents, sentVecs, scores, secIDs = [], [], [], []
    with open('/kaggle/input/graph-structured-vlsp-ds/wordVec/train/train_' + str(cluster) + '.pkl', 'rb') as fp:
        clusTree = pickle.load(fp)
    doc_endsec, secnum, sentnum = divideIntoSections_lda()
    doc_sec_mask = np.zeros((len(clusTree['docs']), secnum))
    sec_sen_mask = np.zeros((secnum, sentnum))
    cursec, cursent = 0, 0
    for d, doc in enumerate(clusTree['docs']):
        doc_sec_mask[d][cursec:doc_endsec[d] + 1] = 1
        cursec = doc_endsec[d] + 1
        for s, sent in enumerate(doc['sents']):
            sents.append(sent['raw_sent'])
            sentVecs.append(meanTokenVecs(sent) + getPositionEncoding(d) + getPositionEncoding(s))
            scores.append(getRouge2(goldenList[cluster], sent['raw_sent'], 'p'))
            sec_sen_mask[sent['section_new'], cursent] = 1
            cursent += 1

    trainGraphs.append(Graph(sents, sentVecs, scores, doc_sec_mask, sec_sen_mask, goldenList[cluster]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [15]:
# trainGraphs = []
# with open('/kaggle/input/vlsp-dataset/train_segmentedSumm.txt', 'r', encoding='utf-8') as f:
#     goldenList = json.load(f)
# for cluster in range(0, 200):
#     print(cluster)
#     sents, sentVecs, scores, secIDs, spanIds = [], [], [], [], []
#     with open('/kaggle/input/vlsp-dataset/train_vec/train_' + str(cluster) + '.pkl', 'rb') as fp:
#         clusTree = pickle.load(fp)
# #     with open('/kaggle/input/vlsp-dataset/train_sent_CLS/train_' + str(cluster) + '.pkl', 'rb') as fp:
# #         sentTree = pickle.load(fp)
#     doc_endsec, secnum, sentnum = divideIntoSections_lda()
#     doc_sec_mask = np.zeros((len(clusTree['docs']), secnum))
#     sec_sen_mask = np.zeros((secnum, sentnum))
#     cursec, cursent = 0, 0
#     for d, doc in enumerate(clusTree['docs']):
#         doc_sec_mask[d][cursec:doc_endsec[d] + 1] = 1
#         cursec = doc_endsec[d] + 1
#         for s, sent in enumerate(doc['sents']):
# #             stt_sent_in_sec = 0
# #             while sec_sen_mask[sent['section_new'], cursent - stt_sent_in_sec - 1] == 1:
# #                 stt_sent_in_sec += 1
#             sents.append(sent['raw_sent'])
#             sentVecs.append(meanTokenVecs(sent) + getPositionEncoding(s) + getPositionEncoding(d))
#             # sentVecs.append(sentTree['docs'][d]['sents'][s]['vector'] + getPositionEncoding(s) + getPositionEncoding(d))
#             scores.append(getRouge2(goldenList[cluster], sent['raw_sent'], 'p'))
#             spanIds.append((d, s, 0))
#             sec_sen_mask[sent['section_new'], cursent] = 1
#             cursent += 1

#     trainGraphs.append(Graph(cluster, sents, sentVecs, spanIds, scores, doc_sec_mask, sec_sen_mask, goldenList[cluster]))

In [16]:
# import pickle

# file_path = '/kaggle/input/vlsp-dataset/train_vec/train_0.pkl'

# with open(file_path, 'rb') as file:
#     data = pickle.load(file)

# print(data)


In [17]:
# with open('/kaggle/input/vlsp-dataset/wordVec/wordVec/test/test_0.pkl', 'rb') as fp:
#     graph = pickle.load(fp)
# spans = graph['docs'][0]['sents'][0]['spans']
# for sp in spans:
#     print((sp['id'], sp['word'], sp['wtype'], sp['deplabel'], sp['children']))

In [18]:
args = {'gpu': 2, 'seed': 42, 'batch_size': 1, 'input': 768, 'hidden': 2048, 'heads': 8,
       'epochs': 60, 'log_every': 20, 'lr': 0.0003, 'dropout': 0.3, 'num_layers': 3}

In [19]:
import random
import os
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(args['seed'])

In [20]:
def train_e2e(train_dataloader, model, optimizer):
    model[0].train()
    model[1].train()
    c_loss, s_loss, loss, batch_num = 0, 0, 0, 0
    print_epo = 20
    rouge2_score = []

    for i, data in enumerate(train_dataloader):
        batch_loss, bc_loss, bs_loss, scores = train_e2e_batch(data, model, optimizer)
        loss += batch_loss
        c_loss += bc_loss
        s_loss += bs_loss
        batch_num += 1

        abs_text = data.golden
        summary_text = get_summary(scores[0], data.sents)
        rouge2_score.append(getRouge2(data.golden, summary_text, 'f'))

        if i % print_epo == 0:
            print("Batch {}, Loss: {}".format(i, loss / batch_num))
            print("Batch {}, C-Loss: {}".format(i, c_loss / batch_num))
            print("Batch {}, S-Loss: {}".format(i, s_loss / batch_num))

    return loss / batch_num, np.mean(rouge2_score)

def train_e2e_batch(data_batch, model, optimizer):
    c_model = model[0]
    s_model = model[1]

    optimizer.zero_grad()
    feature = data_batch.feature.unsqueeze(0)
    adj = data_batch.adj.unsqueeze(0)
    docnum = data_batch.docnum
    secnum = data_batch.secnum
    labels = data_batch.score_onehot.unsqueeze(0)
    scores = data_batch.score.unsqueeze(0).float()
    goldenVec = data_batch.goldenVec

    pg = c_model(feature.cuda(), adj.cuda(), docnum, secnum)
    x = s_model(pg.cuda(), adj.cuda(), docnum, secnum)

    s_loss = F.binary_cross_entropy_with_logits(x.squeeze(-1), labels.cuda(), pos_weight=torch.tensor(10).cuda())
    pg = pg.squeeze(0)
    infonce = InfoNCE(tau=0.2)

    mask = torch.zeros(1, feature.shape[1])
    mask[:, :-docnum-secnum-1] = labels
    mask[:, -docnum-secnum-1:] = 1
    neg_mask = 1 - mask
    c_loss = infonce(goldenVec.cuda(), pg, mask.cuda(), neg_mask.cuda())

    loss = s_loss + 1. * c_loss
    # loss = s_loss
    loss.backward()
    optimizer.step()

    return loss.data, c_loss.data, s_loss.data, torch.sigmoid(x.squeeze(-1))

def val_e2e(val_dataloader, model, mode='val', max_word_num=160, sent_num=0):
    model[0].eval()
    model[1].eval()
    loss, c_loss, s_loss = 0,0,0
    batch_num = 0
    rouge2_score = []

    all_summaries = []
    all_gt = []
    for i, data in enumerate(val_dataloader):
        cur_loss, c_loss_b, s_loss_b, scores = val_e2e_batch(data, model)
        loss += cur_loss
        c_loss += c_loss_b
        s_loss += s_loss_b

        abs_text = data.golden
        summary_text = get_summary(scores[0], data.sents, max_word_num, sent_num)
        all_gt.append(data.golden)
        all_summaries.append(summary_text)
        rouge2_score.append(getRouge2(data.golden, summary_text, 'f'))
        batch_num += 1

    rouge2_score_mean = np.mean(rouge2_score)
    loss = loss / batch_num
    c_loss /= batch_num
    s_loss /= batch_num

    if mode != 'val':
        return rouge2_score_mean, all_summaries, all_gt, rouge2_score
    return rouge2_score_mean, loss, c_loss, s_loss


def val_e2e_batch(data_batch, model):
    c_model = model[0]
    s_model = model[1]
    feature = data_batch.feature.unsqueeze(0)
    adj = data_batch.adj.unsqueeze(0)
    docnum = data_batch.docnum
    secnum = data_batch.secnum
    labels = data_batch.score_onehot.unsqueeze(0)
    scores = data_batch.score.unsqueeze(0).float()
    goldenVec = data_batch.goldenVec

    with torch.no_grad():
        pg = c_model(feature.cuda(), adj.cuda(), docnum, secnum)
        x = s_model(pg.cuda(), adj.cuda(), docnum, secnum)

        pg = pg.squeeze(0)
        infonce = InfoNCE(tau=0.2)

        mask = torch.zeros(1, feature.shape[1])
        mask[:, :-docnum-secnum-1] = labels
        mask[:, -1] = 1
        neg_mask = 1 - mask
        c_loss = infonce(goldenVec.cuda(), pg, mask.cuda(), neg_mask.cuda())
        s_loss = F.binary_cross_entropy_with_logits(x.squeeze(-1), labels.cuda(), pos_weight=torch.tensor(10))
#         s_loss = F.mse_loss(torch.sigmoid(x.squeeze(-1)), scores)

        loss = c_loss * 1. + s_loss
        # loss = s_loss
        scores = torch.sigmoid(x.squeeze(-1))
    return loss.data, c_loss.data, s_loss.data, scores

def get_summary(scores, sents, max_word_num=160, sent_num=0):
    ranked_score_idxs = torch.argsort(scores, dim=0, descending=True)
    wordCnt = 0
    summSentIDList = []
    for i in ranked_score_idxs:
        if wordCnt >= max_word_num and sent_num == 0: break
        elif sent_num > 0 and len(summSentIDList) == sent_num: break
        s = sents[i]

        replicated = False
        for chosedID in summSentIDList:
            if getRouge2(sents[chosedID], s, 'p') >= 0.45:
                replicated = True
                break
        if replicated: continue

        wordCnt += len(s.split(' '))
        summSentIDList.append(i)
    summSentIDList = sorted(summSentIDList)
    return ' '.join([s for i, s in enumerate(sents) if i in summSentIDList])

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [22]:
c_graph_encoder = StepWiseGraphConvLayer(in_dim=768, out_dim=args['hidden'], hid_dim=args['hidden'],
                                         dropout_p=args['dropout'], act=nn.LeakyReLU(), nheads=8, iter=1).to(device) 
s_graph_encoder = StepWiseGraphConvLayer(in_dim=768, out_dim=args['hidden'], hid_dim=args['hidden'],
                                         dropout_p=args['dropout'], act=nn.LeakyReLU(), nheads=8, iter=1).to(device)
contrast_filter = Contrast_Encoder(c_graph_encoder, args['hidden'], dropout_p=args['dropout']).to(device)
summarization_encoder = End2End_Encoder(s_graph_encoder, 768, args['hidden'], args['dropout']).to(device)

In [23]:
trainset, valset = trainGraphs, testGraphs[:100]

In [24]:
optimizer = torch.optim.Adam([ {'params': summarization_encoder.parameters()},
                            {'params': contrast_filter.parameters()}], lr=args['lr'], weight_decay=1e-5)

In [25]:
model_save_root_path = '/kaggle/working/'
c_patient, s_patient = 30,30
best_r2, best_c_loss, best_s_loss = 0, 10000, 10000
history = {'loss': [], 'val_loss': []}

for i in range(args['epochs']):
    print("Epoch {}".format(i))
    random.shuffle(trainset)

    if c_patient < 0:
        for p in contrast_filter.parameters():
            p.requires_grad = False
        print("Stop Training Contrast")

    model = [contrast_filter, summarization_encoder]
    loss, rouge2_score = train_e2e(trainset, model, optimizer)
    history['loss'].append(loss)
    print("At Epoch {}, Train Loss: {}, R2 score: {}".format(i, loss, rouge2_score))
    torch.cuda.empty_cache()

    rouge2_score, loss, c_loss, s_loss = val_e2e(valset, model)
    torch.cuda.empty_cache()
    history['val_loss'].append(loss)
    print("At Epoch {}, Val Loss: {}, Val CLoss: {}, Val SLoss: {},Val R2: {}".format(i, loss, c_loss, s_loss, rouge2_score))
    if rouge2_score > best_r2:
        model_save_path = os.path.join(model_save_root_path, "e_{}_{}.mdl".format(i, rouge2_score))
        torch.save(summarization_encoder.state_dict(), model_save_path)

        model_save_path = os.path.join(model_save_root_path, "c_{}_{}.mdl".format(i, rouge2_score))
        torch.save(contrast_filter.state_dict(), model_save_path)
        best_r2 = rouge2_score
        print("Epoch {} Has best R2 Score of {}, saved Model to {}".format(i, best_r2, model_save_path))
#     if c_loss < best_c_loss and c_patient >= 0:
#         best_c_loss = c_loss
#         c_patient = 30
#     else:
#         c_patient -= 1

Epoch 0
Batch 0, Loss: 3.171091079711914
Batch 0, C-Loss: 1.1061503887176514
Batch 0, S-Loss: 2.0649406909942627
Batch 20, Loss: 2.8123083114624023
Batch 20, C-Loss: 1.3621066808700562
Batch 20, S-Loss: 1.4502017498016357
Batch 40, Loss: 2.705322265625
Batch 40, C-Loss: 1.1985633373260498
Batch 40, S-Loss: 1.5067590475082397
Batch 60, Loss: 2.697244882583618
Batch 60, C-Loss: 1.128234624862671
Batch 60, S-Loss: 1.5690101385116577
Batch 80, Loss: 2.643364906311035
Batch 80, C-Loss: 1.0907566547393799
Batch 80, S-Loss: 1.5526081323623657
Batch 100, Loss: 2.571589469909668
Batch 100, C-Loss: 1.0052095651626587
Batch 100, S-Loss: 1.566379189491272
Batch 120, Loss: 2.5651724338531494
Batch 120, C-Loss: 0.9511628150939941
Batch 120, S-Loss: 1.6140083074569702
Batch 140, Loss: 2.4770445823669434
Batch 140, C-Loss: 0.8901317715644836
Batch 140, S-Loss: 1.5869115591049194
Batch 160, Loss: 2.480642318725586
Batch 160, C-Loss: 0.8782008290290833
Batch 160, S-Loss: 1.6024409532546997
Batch 180, Lo

In [26]:
# model_save_root_path = './models'
# c_patient = 30
# best_r2, best_c_loss, best_s_loss = 0, 10000, 10000
# history = {'loss': [], 'val_loss': []}

# for i in range(args['epochs']):
#     print("Epoch {}".format(i))
#     random.shuffle(trainset)

#     if c_patient < 0:
#         for p in contrast_filter.parameters():
#             p.requires_grad = False
#         print("Stop Training Contrast")

#     model = [contrast_filter, summarization_encoder]
#     loss = train_e2e(trainset, model, optimizer)
#     history['loss'].append(loss)
#     print("At Epoch {}, Train Loss: {}".format(i, loss))
#     #     torch.cuda.empty_cache()

#     rouge2_score, loss, c_loss, s_loss = val_e2e(valset, model)
#     #     torch.cuda.empty_cache()
#     history['val_loss'].append(loss)
#     print("At Epoch {}, Val Loss: {}, Val CLoss: {}, Val SLoss: {},Val R1: {}".format(i, loss, c_loss, s_loss,
#                                                                                       rouge2_score))
#     if rouge2_score > best_r2:
#         model_save_path = os.path.join(model_save_root_path, "e_{}_{}.mdl".format(i, s_loss))
#         torch.save(summarization_encoder.state_dict(), model_save_path)

#         model_save_path = os.path.join(model_save_root_path, "c_{}_{}.mdl".format(i, c_loss))
#         torch.save(contrast_filter.state_dict(), model_save_path)
#         best_r2 = rouge2_score
#         print("Epoch {} Has best R1 Score of {}, saved Model to {}".format(i, best_r2, model_save_path))
#     if c_loss < best_c_loss and c_patient >= 0:
#         best_c_loss = c_loss
#         c_patient = 30
#     else:
#         c_patient -= 1